In [6]:
from SuperCNN import CNN

from SuperCNN import Parametros

from Auxiliares import takeinputs


In [7]:
val, pix, qcmval, qcmpix, pixelsconv, qcmpixconv = takeinputs()

convlay = [(1, "input"), (3, "relu")]

lay = [(64, "sigmoid"), (10, "softmax")]

parametros = Parametros(pix=pix, vales=val, qcmpix=qcmpix, qcmval=qcmval, infolay=lay, infoconvlay=convlay, padding=0, convrapide=False)

In [8]:
g = CNN(parametros)

g.train()


Percentage: 0.0
Percentage: 1.6666666666666667
Percentage: 3.3333333333333335
Percentage: 5.0
Percentage: 6.666666666666667
Percentage: 8.333333333333334
Percentage: 10.0
Percentage: 11.666666666666666
Percentage: 13.333333333333334
Percentage: 15.0
Percentage: 16.666666666666668
Percentage: 18.333333333333332
Percentage: 20.0
Percentage: 21.666666666666668
Percentage: 23.333333333333332
Percentage: 25.0
Percentage: 26.666666666666668
Percentage: 28.333333333333332
Percentage: 30.0
Percentage: 31.666666666666668
Percentage: 33.333333333333336
Percentage: 35.0
Percentage: 36.666666666666664
Percentage: 38.333333333333336
Percentage: 40.0
Percentage: 41.666666666666664
Percentage: 43.333333333333336
Percentage: 45.0
Percentage: 46.666666666666664
Percentage: 48.333333333333336
Percentage: 50.0
Percentage: 51.666666666666664
Percentage: 53.333333333333336
Percentage: 55.0
Percentage: 56.666666666666664
Percentage: 58.333333333333336
Percentage: 60.0
Percentage: 61.666666666666664
Percenta

In [9]:
print(g.tauxlent())

90.78
